<a href="https://colab.research.google.com/github/Silpa12345/Deep_learning/blob/main/IMERGSpecific.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
TARGET = "2019-05-21"  # <-- your trial date

# Paths
ELSUS_PATH = "/content/drive/MyDrive/Project-Landslide/ELSUS/elsus_v2.asc"
EVENTS_CSV = "/content/drive/MyDrive/Project-Landslide/ELSUS/events_glc_europe_clean.csv"
DL_DIR     = "/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4"
RAIN_DIR   = "/content/drive/MyDrive/Project-Landslide/ELSUS/rain_features_TARGET"
OUTDIR     = "/content/drive/MyDrive/Project-Landslide/ELSUS/outputs_trial_20190521"

import os
os.makedirs(DL_DIR, exist_ok=True)
os.makedirs(RAIN_DIR, exist_ok=True)
os.makedirs(OUTDIR, exist_ok=True)


In [2]:
from datetime import datetime, timedelta, date

MODE = "7d"  # options: "1d", "3d", "7d"

d = datetime.strptime(TARGET, "%Y-%m-%d").date()
if MODE == "1d":
    need_days = {d}
elif MODE == "3d":
    need_days = {d - timedelta(days=k) for k in range(0, 3)}    # d, d-1, d-2
elif MODE == "7d":
    need_days = {d - timedelta(days=k) for k in range(0, 7)}    # d, d-1, ... d-6
else:
    raise ValueError("MODE must be one of: '1d', '3d', '7d'")

print("IMERG days to download:", sorted(need_days))


IMERG days to download: [datetime.date(2019, 5, 15), datetime.date(2019, 5, 16), datetime.date(2019, 5, 17), datetime.date(2019, 5, 18), datetime.date(2019, 5, 19), datetime.date(2019, 5, 20), datetime.date(2019, 5, 21)]


In [3]:
!pip -q install earthaccess

import earthaccess as ea
ea.login(strategy="interactive")  # authorize once in the popup

downloaded = []
for day in sorted(need_days):
    s = day.strftime("%Y-%m-%d")
    e = s
    granules = ea.search_data(
        short_name="GPM_3IMERGDF",  # IMERG Daily Final
        temporal=(s, e),
        provider="GES_DISC",
        # version="07",  # uncomment if needed; leave off if it returns 0
    )
    if not granules:
        print("No granules for", s)
        continue
    paths = ea.download(granules, DL_DIR)
    downloaded.extend(paths)

print("Downloaded:", len(downloaded))
print(downloaded[:3])


Enter your Earthdata Login username: silpa_kuriakose
Enter your Earthdata password: ··········


QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

...

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Downloaded: 7
['/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4/3B-DAY.MS.MRG.3IMERG.20190515-S000000-E235959.V07B.nc4', '/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4/3B-DAY.MS.MRG.3IMERG.20190516-S000000-E235959.V07B.nc4', '/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4/3B-DAY.MS.MRG.3IMERG.20190517-S000000-E235959.V07B.nc4']


In [4]:
!pip -q install netCDF4 h5netcdf rioxarray xarray rasterio

import glob, re, numpy as np, rasterio
from rasterio.crs import CRS
import xarray as xr, rioxarray as rxr
from rasterio.enums import Resampling
from collections import deque

# Build nc_files list for just the days we targeted
nc_files = []
for day in sorted(need_days):
    ymd = day.strftime("%Y%m%d")
    nc_files += glob.glob(f"{DL_DIR}/**/*{ymd}*.nc4", recursive=True)
nc_files = sorted(set(nc_files))
print("nc_files to process:", len(nc_files))
print(nc_files[:3])

# Prepare ELSUS profile + template grid
with rasterio.open(ELSUS_PATH) as src:
    prof = src.profile.copy()
    if src.crs is None:
        prof["crs"] = CRS.from_epsg(3035)  # ETRS89 / LAEA Europe (ELSUS v2)
    else:
        prof["crs"] = src.crs
prof.update(count=1, dtype="float32", nodata=np.nan)
template_transform = prof["transform"]
template_shape = (prof["height"], prof["width"])
template_crs = prof["crs"]

# Helpers
def _open_nc(nc_path):
    for engine in ("netcdf4", "h5netcdf", "scipy"):
        try:
            return xr.open_dataset(nc_path, engine=engine, decode_times=True, mask_and_scale=True)
        except Exception:
            pass
    raise RuntimeError(f"Failed to open {nc_path}")

def date_from_filename_nc4(path):
    m = re.search(r"(20\d{6})", os.path.basename(path))
    if not m: raise ValueError(f"No YYYYMMDD in {os.path.basename(path)}")
    from datetime import datetime
    return datetime.strptime(m.group(1), "%Y%m%d").date()

def read_imerg_nc4_to_template(nc_path):
    ds = _open_nc(nc_path)
    if "latitude" in ds.dims or "latitude" in ds.coords: ds = ds.rename({"latitude":"lat"})
    if "longitude" in ds.dims or "longitude" in ds.coords: ds = ds.rename({"longitude":"lon"})
    var = "precipitationCal" if "precipitationCal" in ds.variables else ("precipitation" if "precipitation" in ds.variables else None)
    assert var is not None, f"No precip var in {os.path.basename(nc_path)}"
    da = ds[var].squeeze()
    if da.dims != ("lat","lon"): da = da.transpose("lat","lon")
    if (da["lon"] > 180).any():
        lon = (((da["lon"] + 180) % 360) - 180)
        da = da.assign_coords(lon=lon).sortby("lon")
    da = da.rio.write_crs("EPSG:4326", inplace=True)
    da = da.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
    return da.rio.reproject(dst_crs=template_crs, transform=template_transform, shape=template_shape, resampling=Resampling.bilinear)

# Process: write 1d always; 3d/7d depending on MODE
buf3, buf7 = deque(maxlen=3), deque(maxlen=7)

for nc in nc_files:
    d = date_from_filename_nc4(nc)
    da = read_imerg_nc4_to_template(nc)
    arr = da.values.astype("float32")

    # 1d
    out1 = os.path.join(RAIN_DIR, f"rain_1d_{d:%Y%m%d}.tif")
    with rasterio.open(out1, "w", **prof) as dst:
        dst.write(arr, 1)

    buf3.append(arr)
    buf7.append(arr)

    if MODE in ("3d","7d") and len(buf3) == 3:
        out3 = os.path.join(RAIN_DIR, f"rain_3d_{d:%Y%m%d}.tif")
        with rasterio.open(out3, "w", **prof) as dst:
            dst.write(np.nansum(buf3, axis=0).astype("float32"), 1)

    if MODE == "7d" and len(buf7) == 7:
        out7 = os.path.join(RAIN_DIR, f"rain_7d_{d:%Y%m%d}.tif")
        with rasterio.open(out7, "w", **prof) as dst:
            dst.write(np.nansum(buf7, axis=0).astype("float32"), 1)

print("Done writing to:", RAIN_DIR)


nc_files to process: 7
['/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4/3B-DAY.MS.MRG.3IMERG.20190515-S000000-E235959.V07B.nc4', '/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4/3B-DAY.MS.MRG.3IMERG.20190516-S000000-E235959.V07B.nc4', '/content/drive/MyDrive/Project-Landslide/ELSUS/imerg_daily_TARGET_nc4/3B-DAY.MS.MRG.3IMERG.20190517-S000000-E235959.V07B.nc4']


Done writing to: /content/drive/MyDrive/Project-Landslide/ELSUS/rain_features_TARGET
